<a href="https://colab.research.google.com/github/saritmaitra/Optimizatiion_problem/blob/main/PortfolioOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os                               
import math                             
import numpy as np                      
import pandas as pd                     
import matplotlib.pyplot as plt        
from matplotlib import cm               
import scipy                            
import cvxpy as cp                      
from mpl_toolkits.mplot3d import Axes3D 
%matplotlib inline
from google.colab import files

In [2]:
uploaded = files.upload()

Saving prices.txt to prices (1).txt


In [3]:
df = pd.read_csv("prices.txt", parse_dates = True, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2521 entries, 2007-01-02 to 2016-12-06
Data columns (total 53 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   AGG       2521 non-null   float64
 1   DBC       2521 non-null   float64
 2   DFE       2521 non-null   float64
 3   DIA       2521 non-null   float64
 4   DXJ       2521 non-null   float64
 5   EEM       2521 non-null   float64
 6   EFA       2521 non-null   float64
 7   EWG       2521 non-null   float64
 8   EWH       2521 non-null   float64
 9   EWI       2521 non-null   float64
 10  EWT       2521 non-null   float64
 11  EWU       2521 non-null   float64
 12  EWW       2521 non-null   float64
 13  EWY       2521 non-null   float64
 14  EWZ       2521 non-null   float64
 15  EZU       2521 non-null   float64
 16  FEZ       2521 non-null   float64
 17  FXI       2521 non-null   float64
 18  GDX       2521 non-null   float64
 19  GLD       2521 non-null   float64
 20  IAU       25

It covers 10 years, from January 2006 to December 2016, and comprises a set of 52 popular exchange traded funds (ETFs) and the US central bank (FED) rate of return

In [4]:
df.shape

(2521, 53)

In [5]:
data = df.reset_index()
data.rename(columns = {'index': 'date'}, inplace=True)
data.tail()

,date,AGG,DBC,DFE,DIA,DXJ,EEM,EFA,EWG,EWH,EWI,EWT,EWU,EWW,EWY,EWZ,EZU,FEZ,FXI,GDX,GLD,IAU,IBB,ITB,IVV,IWD,IWM,IYR,KBE,KRE,LQD,OIL,SDS,SH,SLV,SPY,USO,VGK,VNQ,VTI,VWO,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP,USDOLLAR
2516,2016-11-30,108.24,15.21,52.17,191.40,49.20,35.50,56.79,24.83,21.17,21.75,30.74,30.06,44.12,53.26,33.49,32.78,31.38,37.59,20.83,111.75,11.29,274.07,27.37,221.53,109.91,131.61,74.85,41.15,52.41,116.84,5.82,15.75,37.31,15.66,220.38,10.93,45.91,80.39,113.78,36.22,33.85,49.94,74.43,22.51,62.41,47.50,50.58,46.75,68.75,81.85,31.10,41.93,10682.421
2517,2016-12-01,107.68,15.49,52.36,191.90,48.61,35.08,56.57,24.68,20.98,21.98,30.41,30.10,43.27,53.46,31.48,32.64,31.31,37.59,20.66,111.54,11.27,269.49,27.01,220.73,110.08,130.97,73.67,41.98,53.46,115.87,6.08,15.87,37.45,15.66,219.57,11.33,45.84,79.21,113.31,35.81,33.53,49.96,74.61,22.90,62.82,46.52,50.25,46.38,68.25,81.89,31.29,42.05,10682.621
2518,2016-12-02,108.02,15.60,52.28,191.74,48.77,35.12,56.71,24.74,20.74,22.13,30.50,30.35,43.22,53.29,31.68,32.69,31.29,37.31,21.37,112.14,11.32,270.46,27.04,220.91,110.05,130.90,74.45,41.64,53.03,116.40,6.19,15.86,37.43,15.83,219.68,11.48,45.99,80.00,113.31,35.77,33.63,49.98,74.83,22.65,62.81,46.69,50.57,46.80,68.41,81.44,32.06,42.03,10682.826
2519,2016-12-05,108.11,15.68,53.05,192.22,49.01,35.41,57.27,25.34,20.82,22.31,30.57,30.57,43.70,53.54,32.12,33.41,32.00,37.26,21.33,111.54,11.26,271.93,27.40,222.19,110.66,133.15,75.04,42.16,53.68,116.65,6.11,15.66,37.21,15.88,221.00,11.35,46.68,80.59,114.10,36.01,34.04,50.37,75.49,22.92,62.76,47.14,50.63,46.86,68.29,82.33,33.21,42.45,10683.036
2520,2016-12-06,108.09,15.61,53.56,192.49,49.36,35.60,57.66,25.56,20.83,23.25,30.90,30.67,44.34,53.79,32.68,33.83,32.52,37.26,21.20,111.43,11.26,274.34,27.86,222.99,111.29,134.59,75.52,42.76,54.50,116.64,6.06,15.55,37.10,15.85,221.70,11.29,47.11,81.11,114.68,36.16,34.37,50.43,75.35,23.12,62.88,47.28,50.78,46.77,68.48,82.52,33.18,42.70,10683.036


## Optimization
### Constants, parameters and variables

In [6]:
# last price for each stock

p = data \
    .drop(['date'], axis=1) \
    .tail(1) \
    .to_numpy()

In [7]:
# weekly returns from 1 January 2016 onwards

r = data[(data['date'].dt.weekday == 4) & (data['date'] >= '2016-01-01')]
r = r.drop(['date'], 1)
r = r.pct_change()

In [8]:
# expected return and covariance matrix

sigma = r.dropna().cov().to_numpy()
mu = r.dropna().mean().to_numpy()

In [9]:
# optimization variable and parameters

x = cp.Variable(shape = df.shape[1], integer=True)
threshold = cp.Parameter(nonneg=True) # maximum portfolio variance
k = cp.Parameter(nonneg=True) # maximum allocation into one stock

In [10]:
# portfolio mean and variance

mean = mu.T*x
variance = cp.quad_form(x, sigma)

## Objective function

Here our objective is to maximize expected portfolio return

In [11]:
objective = cp.Maximize(mean)

## Constraints

In [12]:
# optimisation constraints
constraints = [x >= 0,                                 # no shorting
    variance <= threshold                   # upper bound on portfolio variance
]
for pi in p:
    constraints = constraints + [pi*x <= k] # upper bound on single-stock allocation

## Solution
Initialize the optimization problem using objective function and constraints

In [13]:
problem = cp.Problem(objective, constraints)

Solve optimization problem for each parameter combination

In [14]:
z_values = []
k_values = np.arange(1000, 5000, 1000)
threshold_values = np.arange(1, 5.5, 0.5)
for threshold_value in threshold_values:
    for k_value in k_values:
        threshold.value = threshold_value
        k.value = k_value
        problem.solve()
        if problem.status != 'optimal': continue
        counts = x.value.round()
        investments = p*counts
        returns = mu@investments[0]
        z_values.append(returns)

## Optimal portfolio
optimal portfolio using highest variance and maximum single-asset allocation

In [17]:
stocks_optimal = (df.columns).values[np.where(counts > 0)]
print('Stocks=>', stocks_optimal); 
counts_optimal = counts[counts > 0]
print('Counts=>', counts_optimal); 
prices_optimal = np.around(p, 2)[0][np.where(counts > 0)]
print('Prices=>', prices_optimal); 
investments_optimal = np.around(investments, 2)[investments > 0]
print('Investments=>', investments_optimal); print()
capital_optimal = np.around(counts_optimal@prices_optimal, 2)
print('Capital=>', capital_optimal); 
risk_optimal = np.around(counts.T@sigma@counts, 2)
print('Risk=>', risk_optimal)
return_optimal = np.around(52*returns/capital_optimal, 3)
print('Return=>', return_optimal); print()

Stocks=> ['DBC' 'EWT' 'GDX' 'KBE' 'SH' 'XLE' 'XLK' 'XME']
Counts=> [ 1.  7.  3.  1.  4. 25. 19. 22.]
Prices=> [15.61 30.9  21.2  42.76 37.1  75.35 47.28 33.18]
Investments=> [  15.61  216.3    63.6    42.76  148.4  1883.75  898.32  729.96]

Capital=> 3998.7
Risk=> 4.99
Return=> 0.519



In [18]:
Stocks = pd.DataFrame(stocks_optimal)
Stocks.rename(columns = {0: 'Stocks_name'}, inplace=True)
Counts = pd.DataFrame(counts_optimal)
Counts.rename(columns = {0: 'Counts'}, inplace=True)
Prices = pd.DataFrame(prices_optimal)
Prices.rename(columns = {0: 'OptimalPrices'}, inplace=True)
Investments = pd.DataFrame(investments_optimal)
Investments.rename(columns = {0: 'OptimalInvestments'}, inplace=True)
pd.concat([Stocks, Counts, Prices, Investments], 1)

,Stocks_name,Counts,OptimalPrices,OptimalInvestments
0,DBC,1.0,15.61,15.61
1,EWT,7.0,30.90,216.30
2,GDX,3.0,21.20,63.60
3,KBE,1.0,42.76,42.76
4,SH,4.0,37.10,148.40
5,XLE,25.0,75.35,1883.75
6,XLK,19.0,47.28,898.32
7,XME,22.0,33.18,729.96


- XLE (energy sector), XLK (technology sector) and XME (metal & mining) are dominating the portfolio with > 80% of total investments. 

- expected return is based on past prices which is not a reasonable indicator of future performance.